# Lab 3 Solving an optimisation problem (TSP) using three local search algorithms
This notebook includes four parts:
1. Problem definition
2. Implementation of Hill Climbing (HC) algorithm and its application to solve the target optimisation problem
3. Implementation of Genetic Algorithm (GA) and its application to solve the target optimisation problem
4. Implementation of Simulated Annealing (SA) algorithm and its application to solve the target optimisation problem

## Part 1 -- Problem definition
### Search problem: Travelling salesman problem (TSP)

In the TSP, an agent must start from a given city, visit every other city exactly once, and return to the start city, while minimizing travel cost. In this example, we consider a region, say Romania, with known city locations.

To define a TSP problem, we need to have the following elements:
1. All cities with their names and locations.
2. Connections between cities, including the distances (weights) between each pair.
3. A neighbour function to generate the neighbouring solution for a given solution.
4. A cost or objective function to assess the quality of a solution."

mount drive for accessing the local files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


move to the directory where the local files are stored

In [ ]:
cd /content/gdrive/MyDrive/IntroAILab/Lab3

/content/gdrive/MyDrive/IntroAILab/Lab3


Import a Python data analysis library, pandas, and give it a short name 'pd'

In [ ]:
<'Add your code here'>

Use `read_csv()` method in `pd` to read a csv file, `problemData.csv`, which contains cities and their locations and save the output into a dataframe, called `df`. Note. The csv file should be in the current directory.

In [ ]:
df = <'Add your code here'>

In [ ]:
df.head()

,LocationName,LocationX,LocationY
0,Arad,91,492
1,Bucharest,400,327
2,Craiova,253,288
3,Drobeta,165,299
4,Eforie,562,293


Create a dictionary, called `locations`, to store the city names (keys) and their respective locations (values).
One element of the dictionary should be like `"Arad": (91, 492)`.

A useful Python function `zip()` is used.
"The zip() function in Python is a built-in utility that combines two or more iterables (like lists, tuples, or strings) element-wise into tuples. Each tuple contains the elements from the corresponding positions of the input iterables. The resulting tuples are collected into an iterator, which can be converted into other data types like lists or tuples." *from ChatGPT 40 mini*

In [ ]:
locations = {}
for x1,x2 in zip(df["LocationName"],zip(df["LocationX"],df["LocationY"])):
    <'Add your code here'>

In [ ]:
locations

{'Arad': (91, 492),
 'Bucharest': (400, 327),
 'Craiova': (253, 288),
 'Drobeta': (165, 299),
 'Eforie': (562, 293),
 'Fagaras': (305, 449),
 'Giurgiu': (375, 270),
 'Hirsova': (534, 350),
 'Iasi': (473, 506),
 'Lugoj': (165, 379),
 'Mehadia': (168, 339),
 'Neamt': (406, 537),
 'Oradea': (131, 571),
 'Pitesti': (320, 368),
 'Rimnicu': (233, 410),
 'Sibiu': (207, 457),
 'Timisoara': (94, 410),
 'Urziceni': (456, 350),
 'Vaslui': (509, 444),
 'Zerind': (108, 531)}

In [4]:
import numpy as np

def define_map(city_location):
    global distances
    city_locations = city_location
    all_cities = []
    distances = {}

    for city in city_locations.keys():
        distances[city] = {}
        all_cities.append(city)
    all_cities.sort()

    for name_1, coordinates_1 in city_locations.items():
            for name_2, coordinates_2 in city_locations.items():
                distances[name_1][name_2] = np.linalg.norm(
                    [coordinates_1[0] - coordinates_2[0], coordinates_1[1] - coordinates_2[1]])
                distances[name_2][name_1] = distances[name_1][name_2]

    return (all_cities, distances)

Call the `define_map()` method using the previously generated dictionary `locations` and save the two outputs in `all_cities` and `distances`, respectively.

In [ ]:
all_cities, distances = <'Add your code here'>

Display the top k elements of the nested dictionary `distances` obtained from calling the `define_map()` method. k is obtained from a user.

In [ ]:
from itertools import islice

num = int(input ("input K as the number of items in a dictionary to display: "))
top_K_items = dict(islice(distances.items(), num))
top_K_items

input K as the number of items in a dictionary to display: 1


{'Arad': {'Arad': 0.0,
  'Bucharest': 350.2941620980858,
  'Craiova': 260.4995201531089,
  'Drobeta': 206.70026608594387,
  'Eforie': 511.31399354995165,
  'Fagaras': 218.27734651126764,
  'Giurgiu': 360.4719129141687,
  'Hirsova': 465.20210661603846,
  'Iasi': 382.25645841502796,
  'Lugoj': 135.07405376311175,
  'Mehadia': 171.283390905248,
  'Neamt': 318.1980515339464,
  'Oradea': 88.54942122905152,
  'Pitesti': 260.4169733331528,
  'Rimnicu': 163.97560794215707,
  'Sibiu': 121.16517651536682,
  'Timisoara': 82.05485969764375,
  'Urziceni': 391.6490776192381,
  'Vaslui': 420.7469548315234,
  'Zerind': 42.5440947723653}}

Define a function `generate_k_neighbouring_paths(path, k)` that generates k neighbouring paths for a given path for the Traveling Salesman Problem. This fucntion is the transition model that is specific to TSP.

A path is a sequence of cities to be visited, without having the start city at the end. k is an integer for the number of neighbouring paths created for a given path, default value is 2.

Follow the pseudo code below to define this function:

Function generate_k_neighbouring_paths(path, k):

    Initialize current_path as a copy of path
    Initialize neighbouring_path as a copy of path
    Initialize an empty list neighbouring_paths

    For i from 1 to k:
        Generate a random integer 'left' between 1 and (length of current_path- 1)
        Generate a random integer 'right' between 1 and (length of current_path - 1)
        If left is greater than right:
            Swap left and right
        Reverse the segment of neighbouring_path from index left to index right (inclusive)
        Append neighbouring_path to neighbouring_paths
        Reset neighbouring_path to a copy of path
    
    Return neighbouring_paths


In [ ]:
import random as rand

def generate_k_neighbouring_paths(path, k=2):
    current_path: int = path
    neighbouring_path: int = path
    neighbouring_paths = [] # define an empty list
    for i in range(1, k):
        left = rand.randint(1, current_path - 1)
        right = rand.randint(1, current_path - 1)
        if left > right:
            # assign left to right
            left = right
    
    return neighbouring_paths


Define a cost function, `cost_fun(path)`, that takes a path as input and returns its total distance as the cost.

In various optimisation problems, this is often referred to as an evaluation function, a fitness function, or an objective function. This function assesses the quality of a given solution in the context of the problem at hand.

In the context of the Travelling Salesman Problem, the 'cost' of a path is the total distance that the salesman would travel by following that path. This includes the distance from the last city in the path back to the start city.

A lower cost indicates a shorter total distance and thus a better solution to the problem.

* The input parameter: `path` is a list of cities representing the order in which the cities are visited.

* The returned value: `total_distance` is the total distance of the path as a float number, including the return trip from the last city in the list to the start city.

Use the nested dictionary 'distances', which is created earlier for storing the distances between each pair of cities, to extrat the distance between two cities for calculating the path cost.

Follow the pseudo code below to define this function:

Function cost_fun(path):

    Initialize total_distance as 0

    For i from 0 to (length of path - 2):
        Set city1 as path[i]
        Set city2 as path[i + 1]
        Add distance between city1 and city2 from distances to total_distance
    
    Set start_city as path[0]
    Set final_city as path[length of path - 1]
    Add distance between start_city and final_city from distances to total_distance

    Return total_distance

In [ ]:
def cost_fun(path):
  total_running_distance = 0
  length_of_path = len(path)
  for i in range(length_of_path):
    city_1 = path[i]
    city_2 = path[i + 1]
    distance_between_cities = abs(city_1 - city_2)
    total_running_distance += distance_between_cities
  
  initial_city = path[0]
  final_city = path[length_of_path - 1]
  distance_difference = initial_city - final_city
  total_running_distance += distance_difference
  
  return total_running_distance

Specify the start city, called `start_city`

In [ ]:
start_city = None # HERE

Here is the end of Part 1 -- problem defition. You need to learn how to define an optimisation problem from this sample problem defition!

## Part 2 -- Implementation of Hill Climbing (HC) algorithm and its application to solve the target optimisation problem

In [ ]:
def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items

In [ ]:
def argmin_random_tie(seq, key):
    """Return a minimum element of seq; break ties at random."""
    return min(shuffled(seq), key=key)

In [ ]:
def hill_climbing(city_list, neighbour_function, cost_fun, times=500, iteration = 1000, start_point = None):
    current = city_list
    iterations = iteration
    count = 0
    while iterations:
        neighbours = neighbour_function(current)
        if not neighbours:
            break
        neighbour = argmin_random_tie(neighbours, key=lambda node: cost_fun(node))
        if count >= times:
            break
        if cost_fun(neighbour) < cost_fun(current):
            current = neighbour
            count = 0
        else:
            count += 1
        iterations -= 1
    if start_point != None:
        current.append(start_point)
    return (current,cost_fun(current))

Define a method, `define_init()` to create an initial solution for the Traveling Salesman Problem (TSP).

For example, if the list of all cities is ["A", "B", "C", "D"] and the start city is "B", this function would return the list ["B", "A", "C", "D"].

This list represents a solution to the TSP where the journey starts at city "B" and then goes to "A", then "C" and "D", and finally goes back to "A".

This initial solution is then used as the starting solution for the search algorithm to find an optimal solution.

Follow the pseudo code below to define this method.

Function define_init(city_list, start_point):

    Create a copy of city_list and name it init_citylist

    Remove start_point from init_citylist

    Insert start_point at the beginning of init_citylist

    Return init_citylist


In [ ]:
def define_init(city_list, start_point):
  <'Add your code here'>

Call the function `define_init(city_list, start_point)` to generate an initial solution, called `init`, using all the cities (`all_cities`) and the start city (`start_city`) in this problem.

In [ ]:
<"Add your code here ">

Define the termination criteria which consists of two parts:
* the max number of iterations (`max_iteration`), say 1000. If the maximum number of iterations is reached, the search will terminate.
* A number of consecutive times (`n_times`) used to judge if the search terminates, say 50,
If the current solutuin has not changed for `n_times` (e.g., `n_times = 50`) consecutive times, the search will terminate.

Note: The termination criteria can affect the final solution.

In [ ]:
<"Add your code here ">

Invoke the the local search algorithm to solve this TSP.

You can call the hill-climbing algorithm function as `hill_climbing(initial-solution, k-neighbour-path-function, cost-fun, number-of-consecutive-times-to-run-before-the-process-terminates, max-number-of-iterations, start-city)` and save the two outputs in two variables, `solution` and `cost`, respectively.

In [ ]:
solution, cost = <"Add your code here ">

In [ ]:
print(solution, cost)

['Arad', 'Zerind', 'Oradea', 'Sibiu', 'Fagaras', 'Neamt', 'Iasi', 'Vaslui', 'Hirsova', 'Eforie', 'Urziceni', 'Bucharest', 'Giurgiu', 'Pitesti', 'Craiova', 'Drobeta', 'Mehadia', 'Rimnicu', 'Lugoj', 'Timisoara', 'Arad'] 1683.453638574369


In [ ]:
init_representation = init
print(init_representation, cost_fun(init))

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind', 'Arad'] 4264.1914235182


## Part 3 -- Implementation of Genetic Algorithm (GA) and its application to solve the target optimisation problem

In [ ]:
from collections import Counter

def test_duplicated(x):
    duplicated = False
    duplicate = dict(Counter(x))
    for key,value in duplicate.items():
        if value > 1:
            duplicated = True
    return duplicated

In [ ]:
import random

def generate_for_Non_Duplication(x, y):
    new_gen1 = x[:]
    new_gen2 = y[:]
    new_gen1.remove(start_point)
    new_gen2.remove(start_point)
    n = len(new_gen1)
    new_gen = []
    while True:
        c = random.randint(0, n)
        new_gen = new_gen1[:c] + new_gen2[c:]
        if test_duplicated(new_gen):
            pass
        else:
            new_gen.insert(0, start_point)
            break
    return new_gen

In [ ]:
import random

def mutate_for_Non_Duplication(x, pmut):
    if random.uniform(0, 1) >= pmut:
        return x
    mutate_population = x[:]
    mutate_population.remove(start_point)
    random.shuffle(mutate_population)
    mutate_population.insert(0, start_point)
    return mutate_population

In [ ]:
import random

def recombine_for_Non_Duplication(x, y):
    new_gen1 = x[:]
    new_gen2 = y[:]
    new_gen1.remove(start_point)
    new_gen2.remove(start_point)
    n = len(new_gen1)
    duplicated = False
    c = random.randrange(0, n)
    new_gen = new_gen1[:c] + new_gen2[c:]
    new_gen.insert(0, start_point)
    duplicated = test_duplicated(new_gen)
    if duplicated:
        new_gen = generate_for_Non_Duplication(x, y)
    return new_gen

Define a a random-sample function that picks from seq weighted by weights, which is used in definition of the select operator, `select()`

In [ ]:
import bisect
def weighted_sampler(seq, weights):

    totals = []
    for w in weights:
        totals.append(w + totals[-1] if totals else w)
    return lambda: seq[bisect.bisect(totals, random.uniform(0, totals[-1]))]

In [ ]:
def select(population, fitness_fn):
    fitnesses = map(fitness_fn, population)
    sampler = weighted_sampler(population, fitnesses)
    return [sampler() for i in range(2)]

Define a method `fitness_threshold()` to return a better individual than a given threshold.

In [ ]:
def fitness_threshold(fitness_fn, f_thres, population):
    if not f_thres:
        return None

    fittest_individual = max(population, key=fitness_fn)
    if fitness_fn(fittest_individual) >= f_thres:
        return fittest_individual

    return None


In [ ]:
import random

def recombine(x, y):
    n = len(x)
    c = random.randrange(0, n)
    return x[:c] + y[c:]

In [ ]:
import random

def mutate(x, gene_pool, pmut):
    if random.uniform(0, 1) >= pmut:
        return x

    n = len(x)
    g = len(gene_pool)
    c = random.randrange(0, n)
    r = random.randrange(0, g)

    new_gene = gene_pool[r]
    return x[:c] + [new_gene] + x[c + 1:]

In [ ]:
def init_population_for_Non_Duplication(pop_number, gene_pool, state_length, start):
    g = len(gene_pool)
    initial_population = gene_pool.copy()
    population = []
    for i in range(pop_number):
        random.shuffle(initial_population)
        initial_population.remove(start)
        initial_population.insert(0,start)
        population.append(initial_population)
        initial_population = gene_pool.copy()
    return population



In [ ]:
def genetic_algorithm(population, fitness_fn, gene_pool, f_thres=None, ngen=None, pmut=0.1, start = None, non_duplication = True):
    global start_point
    start_point = start
    new_population=[]
    if non_duplication:
        for generation in range(ngen):
            for i in range(len(population)):
               individual1, individual2 = select(population, fitness_fn);
               individual_recombined = recombine_for_Non_Duplication(individual1, individual2)
               individual_final = mutate_for_Non_Duplication(individual_recombined, pmut)
               new_population.append(individual_final)

            population = new_population

           # population = [mutate_for_Non_Duplication(recombine_for_Non_Duplication(*select(population, fitness_fn)), gene_pool, pmut) for i in range(len(population))]

            # stores the fittest individual genome in the current population
            current_best = max(population, key = fitness_fn)
            print("Generation: {}\t\tFitness: {}\r".format(str(generation),fitness_fn(current_best)))

            # compare the fitness of the current best individual to f_thres
            fittest_individual = fitness_threshold(fitness_fn, f_thres, population)

            # if fitness is greater than or equal to f_thres, we terminate the algorithm
            if fittest_individual:
                return fittest_individual, generation
    else:
        for generation in range(ngen):
            population = [mutate(recombine(*select(population, fitness_fn)), gene_pool, pmut) for i in range(len(population))]
            # stores the individual genome with the highest fitness in the current population
            current_best = max(population, key = fitness_fn)
            print(f'Current best: {current_best}\t\tGeneration: {str(generation)}\t\tFitness: {fitness_fn(current_best)}\r', end='')

            # compare the fitness of the current best individual to f_thres
            fittest_individual = fitness_threshold(fitness_fn, f_thres, population)
            print(fittest_individual)

            # if fitness of the best individual is greater than or equal to f_thres, we terminate the algorithm
            if fittest_individual:
                return fittest_individual, generation

    return max(population, key=fitness_fn) , generation # for using a fitness function, the smaller value, the better


Define the fitness function, `fitness_fun()`, as the difference between a big positive number, called `cap`, and the actual cost, which is the path cost using `cost_fun(path)`. The big number can be close infinit to make sure the fitness function has positive values.

In [ ]:
cap = float(input ("Provide the cap value for making fitness function produce positive values. It should be a very big number. Suggested value is 5000"))
def fitness_fun(path):
<'Add your code here'>

Create a gene pool using all the cities (`all_cities`), called `gene_pool`, which will be used to form solutions.

In [ ]:
<'Add your code here'>

Specify the maximum population size, called `max_population`. This parameter is problem specific. In this example, we set it to be 100.

In [ ]:
<'Add your code here'>

Use the method `init_population_for_Non_Duplication(size-population, gene-pool, number-of-elements-in-gene-pool, start_city)`, to generate the intial population for the given maximum population size, called `population`.

In [ ]:
population = <'Add your code here'>

In [ ]:
print(len(population))

10


Define the mutation rate (`mutation_rate`). This parameter is typically quite small. In this case, we'll use 0.05 to speed up the search process.

Note. This parameter is problem specific.

In [ ]:
<'Add your code here'>

Define the termination criteria to control when the algorithm should stop.

In this case, we will use two criteria:
* the maximum number of generations (`max_gen`), say 1000, and
* the threshold of the fitness function (`f_thres`), say 4000.

The algorithm will stop when either the maximum number of generations is reached or the fitness function value exceeds the threshold.

In [ ]:
<'Add your code here'>

Apply the Genetic Algorithm to solve the problem.

Call the function using `genetic_algorithm(population, fitness-function, gene-pool, fitness-function-threshold, max-number-generation-to-run, mutation-rate, start-point)`and save the outputs in two variables, `solution` and `generations`, respectively.

The function will also display the search results at each step, including the generation number and the corresponding fitness function value.

In [ ]:
solution, cost = <'Add your code here'>

In [ ]:
print("The best solution is: \n{} and \nThe cost is: \n{}".format(solution, cost_fun(solution)))

The best solution is: 
['Arad', 'Neamt', 'Hirsova', 'Urziceni', 'Rimnicu', 'Iasi', 'Fagaras', 'Craiova', 'Giurgiu', 'Pitesti', 'Lugoj', 'Bucharest', 'Vaslui', 'Eforie', 'Drobeta', 'Sibiu', 'Oradea', 'Zerind', 'Mehadia', 'Timisoara'] and 
The cost is: 
3539.983850867764


Display the best solution from the initial population and its corresponding cost for comparison with the final result.

In [ ]:
def shuffled(iterable):
    """Randomly shuffle a copy of iterable."""
    items = list(iterable)
    random.shuffle(items)
    return items

In [ ]:
def argmin_random_tie(seq, key):
    """Return a minimum element of seq; break ties at random."""
    return min(shuffled(seq), key=key)

In [ ]:
evaluation_fn = lambda x: cost_fun(x)
init_representation = argmin_random_tie(population, key= evaluation_fn)
print("The initial solution is:\n{}, and \nThe cost is: \n{}".format(init_representation, cost_fun(init_representation)))

The initial solution is:
['Arad', 'Oradea', 'Iasi', 'Sibiu', 'Eforie', 'Giurgiu', 'Urziceni', 'Hirsova', 'Craiova', 'Lugoj', 'Fagaras', 'Mehadia', 'Drobeta', 'Zerind', 'Neamt', 'Timisoara', 'Pitesti', 'Bucharest', 'Rimnicu', 'Vaslui', 'Arad', 'Arad', 'Arad'], and 
The cost is: 
4344.03830914048


## Part 4 -- Implementation of Simulated Annealing (SA) algorithm and its application to solve the target optimisation problem

In [ ]:
import random

def probability(p):
    """Return true with probability p."""
    return p > random.uniform(0.0, 1.0)

In [ ]:
import numpy as np
import sys

def simulated_annealing(init, energy, neighbor, schedule, start_point = None):
    current = init
    for t in range(sys.maxsize):
        T = schedule()(t)
        if T == 0:
            break
        neighbors = neighbor(current)
        if not neighbors:
            break
        next_choice = random.choice(neighbors)
        delta_e =  energy(next_choice) - energy(current)
        if delta_e < 0 or probability(np.exp(- delta_e / T)):
            current = next_choice
    if start_point != None:
        current.append(start_point)
    return (current,energy(current))

Define a function `define_init()` to create an initial solution based on all the cities given and the start city for the Traveling Salesman Problem (TSP).

For example, if the list of all cities was ["A", "B", "C", "D"] and the start city was "B", this function would return the list ["B", "A", "C", "D"]. This list represents a solution to the TSP where the journey starts at city "B" and then goes to "A", then "C", and finally "D".

This initial solution is then used as the starting point for the local search algorithm to find an optimal solution.

**Follow the pseudo code below to define this method.**

Function define_init(city_list, start_point):

    Create a copy of city_list and name it init_citylist

    Remove start_point from init_citylist

    Insert start_point at the beginning of init_citylist

    Return init_citylist


In [ ]:
def define_init(city_list, start_point):
  <'Add your code here'>

Call the function `define_init(city_list, start_point)` to generate an initial solution (called `init`) using all the cities (`all_cities`) and the start city (`start_city`) in this problem.

In [ ]:
init = <'Add your code here'>

Define the annealing schedule which represents how the temperature changes over time. This function is problem specific.

One possible schedule function for simulated annealing is as follows:

$f(T) = k*exp^{(-lam*T)}$ if $T < limit$, otherwise $f(T) = 0$, where $T$ is temperature variable, $k$ is a coefficient (you can try 20 in this problem),  $exp()$ is the exponential function which is defined in **Numpy**, $limit$ is a constant (you can use $limit$ = 10000) and $lam$ is a coefficient (you can try $lam$ = 0.005).

Use a Python `lamda` function to define this schedule function.
* A Python `lambda` function is a small anonymous function. It can take any number of arguments, but can only have one expression, e.g., $x$ = $lamda$ $a$ : $a + 10$ and $x$ = $lamda$ $a$, $b$, $c$ : $a + b + c$
* The following example of `lambda` function defines a multiple-value function, $y=x$, $x>0$; $y=0$, $x<=0$, $y$=$lamda$ $x$: $x$ if x >0 else 0

In [ ]:
def exp_schedule(k=20, lam=0.005, limit=10000):
   <"Add your code here">

Define the energy/cost function which takes in a state/solution as the input and returns a numeric number. In this problem, the cost function is the total distance to be covered by the Traveling Salesman for the given solution.

**Follow the pseudo code below to define this fucntion.**

Function energy(state):

    Set cost as 0
    
    For i from 0 to (length of state - 2):
        Add distance between state[i] and state[i + 1] from distances to cost
    
    Add distance between state[0] and state[length of state - 1] from distances to cost
    
    Return cost


In [ ]:
def energy(state):
<'Add your code here'>

Invoke the simulated annealing algorithm to solve this given problem.
Call the algorithm using `simulated_annealing(initial-solution, energy-function, neighbour-function, annealing-schedule, start-city)` and save the outputs in two variables, `solution` and `generations`, respectively.

In [ ]:
solution, cost = <"Add your code here">

In [ ]:
# Answer
solution, cost = simulated_annealing(init, energy, generate_k_neighbouring_paths, exp_schedule, start_city)

In [ ]:
print(solution, cost)

['Arad', 'Timisoara', 'Lugoj', 'Mehadia', 'Drobeta', 'Craiova', 'Pitesti', 'Giurgiu', 'Bucharest', 'Urziceni', 'Eforie', 'Hirsova', 'Vaslui', 'Iasi', 'Neamt', 'Fagaras', 'Rimnicu', 'Sibiu', 'Oradea', 'Zerind', 'Arad'] 1589.8433308050924


In [ ]:
init_representation = init
print(init_representation, cost_fun(init))

['Arad', 'Bucharest', 'Craiova', 'Drobeta', 'Eforie', 'Fagaras', 'Giurgiu', 'Hirsova', 'Iasi', 'Lugoj', 'Mehadia', 'Neamt', 'Oradea', 'Pitesti', 'Rimnicu', 'Sibiu', 'Timisoara', 'Urziceni', 'Vaslui', 'Zerind'] 4264.1914235182
